In [14]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('./data/ts_output.parquet.gzip')

In [3]:
df.head()

,ds,yhat,trend,yhat_lower,yhat_upper,item
0,2011-01-29,2.335858,1.962375,-0.003809,4.869013,ACCESORIES_1_015_BOS_1
1,2011-01-30,2.252424,1.962527,-0.217628,4.570816,ACCESORIES_1_015_BOS_1
2,2011-01-31,1.655296,1.962679,-0.695771,4.106503,ACCESORIES_1_015_BOS_1
3,2011-02-01,1.691496,1.962831,-0.526323,4.092190,ACCESORIES_1_015_BOS_1
4,2011-02-02,1.734802,1.962983,-0.652782,3.960646,ACCESORIES_1_015_BOS_1


In [4]:
df['store'] = df['item'].str[-5:]
df['region'] = df['store'].str[0:3]
df['category'] = df['item'].str[:-12]
df.head()

,ds,yhat,trend,yhat_lower,yhat_upper,item,store,region,category
0,2011-01-29,2.335858,1.962375,-0.003809,4.869013,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
1,2011-01-30,2.252424,1.962527,-0.217628,4.570816,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
2,2011-01-31,1.655296,1.962679,-0.695771,4.106503,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
3,2011-02-01,1.691496,1.962831,-0.526323,4.092190,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
4,2011-02-02,1.734802,1.962983,-0.652782,3.960646,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES


In [5]:
list_col = ['store', 'region', 'category']
for i in list_col:
    print(f'Columna {i}')
    print(df[i].value_counts())
    print('--------------')

Columna store
store
PHI_3    247933
BOS_2    247702
BOS_1    247575
NYC_1    247010
NYC_3    246668
BOS_3    243933
NYC_4    243871
PHI_2    243762
PHI_1    238845
NYC_2    237454
Name: count, dtype: int64
--------------
Columna region
region
NYC    975003
BOS    739210
PHI    730540
Name: count, dtype: int64
--------------
Columna category
category
SUPERMARKET      1908207
HOME_&_GARDEN     290725
ACCESORIES        245821
Name: count, dtype: int64
--------------


In [6]:
df.shape

(2444753, 9)

In [7]:
# Redondeamos a la unidad
df['yhat'] = df['yhat'].round(0)

# Cambiamos a entero
df['yhat'] = df['yhat'].astype(int)

# Renombramos columna
df.rename(columns={'yhat': 'sales'}, inplace=True)

df.head()

,ds,sales,trend,yhat_lower,yhat_upper,item,store,region,category
0,2011-01-29,2,1.962375,-0.003809,4.869013,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
1,2011-01-30,2,1.962527,-0.217628,4.570816,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
2,2011-01-31,2,1.962679,-0.695771,4.106503,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
3,2011-02-01,2,1.962831,-0.526323,4.092190,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
4,2011-02-02,2,1.962983,-0.652782,3.960646,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES


In [8]:
df.columns.to_list()

['ds',
 'sales',
 'trend',
 'yhat_lower',
 'yhat_upper',
 'item',
 'store',
 'region',
 'category']

In [9]:
df = df[['ds',
 'sales',
 'item',
 'store',
 'region',
 'category']]

In [12]:
df.head()

,date,sales,item,store,region,category
0,2011-01-29,2,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
1,2011-01-30,2,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
2,2011-01-31,2,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
3,2011-02-01,2,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES
4,2011-02-02,2,ACCESORIES_1_015_BOS_1,BOS_1,BOS,ACCESORIES


In [11]:
# Renombramos columna
df.rename(columns={'ds': 'date'}, inplace=True)

In [15]:
df['sales'] = df['sales'].apply(lambda x: 0 if x < 0 else x)

In [17]:
df['sales'].describe()

count    2.444753e+06
mean     8.228556e+00
std      1.014046e+01
min      0.000000e+00
25%      3.000000e+00
50%      6.000000e+00
75%      1.000000e+01
max      2.770000e+02
Name: sales, dtype: float64

In [18]:
df.to_parquet('./data/forecast_data.parquet.gzip')

In [23]:
df_filter = df[df['store'] == 'NYC_1']

In [24]:
# Obteniendo los items únicos
items = df_filter['item'].unique()

# Generando datos aleatorios para el stock
np.random.seed(0)  # Para reproducibilidad de los resultados
stocks = np.random.randint(0, 120, size=len(items))

# Creando el DataFrame
df_stock_nyc_1 = pd.DataFrame({
    "item": items,
    "stock": stocks
})

In [25]:
df_stock_nyc_1.describe()

,stock
count,136.000000
mean,58.169118
std,34.368494
min,0.000000
25%,32.000000
50%,57.500000
75%,85.500000
max,119.000000


In [26]:
df_stock_nyc_1.to_csv('./data/df_stock.csv')

In [27]:
df_stock_nyc_1.to_csv('C:\\Users\\mario.segura\\Downloads\\df_stocks.csv')

In [28]:
df_stock_nyc_1.set_index('item')

,stock
item,
ACCESORIES_1_015_NYC_1,44
ACCESORIES_1_067_NYC_1,47
ACCESORIES_1_178_NYC_1,117
ACCESORIES_1_234_NYC_1,64
ACCESORIES_1_244_NYC_1,67
...,...
SUPERMARKET_3_800_NYC_1,34
SUPERMARKET_3_804_NYC_1,48
SUPERMARKET_3_808_NYC_1,93
